![gpt2-llama2-llama3](https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/gpt-to-llama/gpt2-to-llama2-llama3.webp?1)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/gpt-to-llama/llama3-to-llama31.webp" width="700px">

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from pretrain import load_model_tokenizer


tokenizer, model = load_model_tokenizer(tokenizer_name='Qwen/Qwen3-0.6B', seq_len=1024, device='mps')

In [ ]:
from pretrain import sample
sample(tokenizer, model, '中国首都是哪?')

In [ ]:
from pretrain import load_dataset
ds = load_dataset(tokenizer, num_proc=30, seq_len=1024)

In [ ]:
from pretrain import train

train(ds, tokenizer, model, 'outputs/buddygpt', 4, 128, True,)